In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
numCells = 1
namexlsfile = 'Fig3I_v3_TRAILonly_'    
flagD = 1
th = 100
ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)
Vn = 1.75E-12
Vc = 5.25E-12

startTime = datetime.now()
print(startTime)

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
spdata[83] = 0.005 # low dose TRAIL
genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
tout_all = np.arange(0,th*3600+1,30)

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module1 = importlib.import_module(model_name)
model1 = model_module1.getModel()
solver1 = model1.getSolver() # Create solver instance
solver1.setMaxSteps = 1e10
model1.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

for nn in range(numCells): 
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata0)))
    xoutG_all[0,:] = genedata0
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model1.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model1.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model1, solver1)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model1.getStateIds()]) 
    condsSDF.to_csv(namexlsfile+'S_'+str(nn)+'.txt',sep="\t")    
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(namexlsfile+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None    
    print(datetime.now() - startTime)
print(datetime.now())


In [ ]:
numCells = 1
namexlsfile = 'Fig3I_v3_ppERKppAktUP_'    
Vn = 1.75E-12
Vc = 5.25E-12

startTime = datetime.now()
print(startTime)

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
spdata[83] = 0.005 # low dose TRAIL
genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
tout_all = np.arange(0,th*3600+1,30)

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module2 = importlib.import_module(model_name)
model2 = model_module2.getModel()
solver2 = model2.getSolver() # Create solver instance
solver2.setMaxSteps = 1e10
model2.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints
model2.setFixedParameterById('k1671',0.01) # 10x == ppMEK binding of ERK
model2.setFixedParameterById('k1674',0.01) # 10x == ppMEK binding of pERK
model2.setFixedParameterById('k1752',0.5) # 10x == PDK1 binding of Akt
model2.setFixedParameterById('k1756',0.5) # 10x == mTORC2 binding of Akt
    
for nn in range(numCells): 
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata0)))
    xoutG_all[0,:] = genedata0    
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model2.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model2, solver2)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model2.getStateIds()]) 
    condsSDF.to_csv(namexlsfile+'S_'+str(nn)+'.txt',sep="\t")    
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(namexlsfile+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None    
    print(datetime.now() - startTime)
print(datetime.now())


In [ ]:
numCells = 1
namexlsfile = 'Fig3I_v3_PumaNoxaUP_'    
Vn = 1.75E-12 
Vc = 5.25E-12 

startTime = datetime.now()
print(startTime)

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
spdata[83] = 0.005 # low dose TRAIL
genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
tout_all = np.arange(0,th*3600+1,30)

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module3 = importlib.import_module(model_name)
model3 = model_module3.getModel()
solver3 = model3.getSolver() # Create solver instance
solver3.setMaxSteps = 1e10
model3.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

for nn in range(numCells): 
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata
    genedata = genedata0
    genedata[334] = genedata[334]*20.0 # Add PUMA mRNA
    genedata[335] = genedata[335]*20.0 # Add NOXA mRNA
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata)))
    xoutG_all[0,:] = genedata
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model3.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model3.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model3, solver3)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model3.getStateIds()]) 
    condsSDF.to_csv(namexlsfile+'S_'+str(nn)+'.txt',sep="\t")    
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(namexlsfile+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None    
    print(datetime.now() - startTime)
print(datetime.now())

In [ ]:
namexls1 = 'Fig3I_v3_TRAILonly_S_0'
namexls2 = 'Fig3I_v3_ppERKppAktUP_S_0'
namexls3 = 'Fig3I_v3_PumaNoxaUP_S_0'

condsS1 = []
condsS2 = []
condsS3 = []

condsDF = pd.read_csv(namexls1+'.txt',header=0,index_col=0,sep="\t")
condsDF = np.double(condsDF.values[:])
condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
condsS1= condsDF
condsDF = None

condsDF = pd.read_csv(namexls2+'.txt',header=0,index_col=0,sep="\t")
condsDF = np.double(condsDF.values[:])
condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
condsS2= condsDF
condsDF = None

condsDF = pd.read_csv(namexls3+'.txt',header=0,index_col=0,sep="\t")
condsDF = np.double(condsDF.values[:])
condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
condsS3= condsDF
condsDF = None

yy1 = np.array(condsS1[:,105])
yy2 = np.array(condsS2[:,105])
yy3 = np.array(condsS3[:,105])
      
tt1 = (np.arange(0,len(condsS1))*30.0/3600.0)
tt2 = (np.arange(0,len(condsS2))*30.0/3600.0)
tt3 = (np.arange(0,len(condsS3))*30.0/3600.0)

plt.figure(figsize=(7, 4))
plt.plot(tt1,yy1,'k',linewidth=4)
plt.plot(tt2,yy2,'b',linewidth=4)
plt.plot(tt3,yy3,'r',linewidth=4)

plt.xlabel('Time (hrs)')
plt.ylabel('cPARP (nM)')
plt.grid(True)
plt.xlim(0, 50)
plt.ylim(0, 400)
plt.legend(['Low TRAIL dose','+ppERK / +ppAKT','+PUMA / +NOXA'])
plt.savefig('Fig3I_v3.png')